In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [3]:
!wget $data

--2025-10-14 20:38:11--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv.1’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.002s  

2025-10-14 20:38:11 (37.3 MB/s) - ‘course_lead_scoring.csv.1’ saved [80876/80876]



In [4]:
df = pd.read_csv('course_lead_scoring.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [5]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [7]:
categorical = list(df.dtypes[df.dtypes == 'object'].index)
categorical

['lead_source', 'industry', 'employment_status', 'location']

In [8]:
numerical = list(df.select_dtypes(include=['int64', 'float64']).columns)
numerical

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [9]:
df[categorical] = df[categorical].fillna('NA')
df[numerical] = df[numerical].fillna(0.0)

In [10]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [11]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [12]:
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

In [13]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

In [14]:
numerical_features = [col for col in numerical if col != 'converted']
numerical_features

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [15]:
corr_matrix = df[numerical_features].corr()
corr_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [16]:
top_corr = corr_matrix.unstack().sort_values(ascending=False)

In [17]:
top_corr = top_corr[top_corr < 1]

In [18]:
most_correlated_pair = top_corr.idxmax()
highest_value = top_corr.max()

print(f"Most correlated features: {most_correlated_pair} with correlation {highest_value:.3f}")

Most correlated features: ('annual_income', 'interaction_count') with correlation 0.027


In [19]:
from sklearn.model_selection import train_test_split

In [20]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [21]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [22]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [23]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [24]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [25]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [26]:
from sklearn.metrics import mutual_info_score

In [27]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, y_train)

In [28]:
mi = df_train[categorical].apply(mutual_info_converted_score)
mi.sort_values(ascending=False).round(2)

lead_source          0.04
employment_status    0.01
industry             0.01
location             0.00
dtype: float64

In [29]:
from sklearn.feature_extraction import DictVectorizer

In [30]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [33]:
y_pred = model.predict_proba(X_val)[:, 1]

In [34]:
convert_decision = (y_pred >= 0.5)

In [35]:
(y_val == convert_decision).mean().round(2)

np.float64(0.7)

In [36]:
original_accuracy = (y_val == convert_decision).mean()
original_accuracy

np.float64(0.6996587030716723)

In [37]:
features_to_test = ['industry', 'employment_status', 'lead_score']
differences = {}

for feature in features_to_test:
    categorical_wo = categorical.copy()
    numerical_features_wo = numerical_features.copy()

    if feature in categorical_wo:
        categorical_wo.remove(feature)

    if feature in numerical_features_wo:
        numerical_features_wo.remove(feature)

    print(f"Without '{feature}':")
    print("Categorical:", categorical_wo)
    print("Numerical:", numerical_features_wo)

    dv = DictVectorizer(sparse=False)

    train_dict_wo = df_train[categorical_wo + numerical_features_wo].to_dict(orient='records')
    X_train_wo = dv.fit_transform(train_dict_wo)

    val_dict_wo = df_val[categorical_wo + numerical_features_wo].to_dict(orient='records')
    X_val_wo = dv.transform(val_dict_wo)

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_wo, y_train)

    y_pred_wo = model.predict_proba(X_val_wo)[:, 1]

    convert_decision_wo = (y_pred_wo >= 0.5)

    accuracy_wo = (y_val == convert_decision_wo).mean()
    print(accuracy_wo)

    difference = accuracy_wo - original_accuracy
    differences[feature] = round(difference, 4)
    print(f"Accuracy difference: {difference:.4f}")


Without 'industry':
Categorical: ['lead_source', 'employment_status', 'location']
Numerical: ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
0.6996587030716723
Accuracy difference: 0.0000
Without 'employment_status':
Categorical: ['lead_source', 'industry', 'location']
Numerical: ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
0.6962457337883959
Accuracy difference: -0.0034
Without 'lead_score':
Categorical: ['lead_source', 'industry', 'employment_status', 'location']
Numerical: ['number_of_courses_viewed', 'annual_income', 'interaction_count']
0.7064846416382252
Accuracy difference: 0.0068


In [38]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [39]:
C_values = [0.01, 0.1, 1, 10, 100]
accuracies = {}

for C in C_values:
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict_proba(X_val)[:, 1]
    convert_decision = (y_pred >= 0.5)
    accuracy = (y_val == convert_decision).mean().round(3)
    accuracies[C] = accuracy

for c, a in accuracies.items():
    print(f"C={c}: accuracy={a}")

best_C = min([c for c in C_values if accuracies[c] == max(accuracies.values())])
print(f"\n Best C: {best_C} with accuracy {accuracies[best_C]}")

C=0.01: accuracy=0.7
C=0.1: accuracy=0.7
C=1: accuracy=0.7
C=10: accuracy=0.7
C=100: accuracy=0.7

 Best C: 0.01 with accuracy 0.7
